In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
fullsample_df = pd.read_csv('fullsample.csv')

In [3]:
fullsample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7395885 entries, 0 to 7395884
Data columns (total 12 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   JOBID      object
 1   STATE      object
 2   BEGIN      object
 3   END        object
 4   REQMEM     object
 5   USEDMEM    object
 6   REQTIME    object
 7   USEDTIME   object
 8   NODES      int64 
 9   CPUS       int64 
 10  PARTITION  object
 11  EXITCODE   object
dtypes: int64(2), object(10)
memory usage: 677.1+ MB


In [4]:
fullsample_df.head()

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
0,30616928,RUNNING,2021-07-31T22:15:00,Unknown,2048Mn,0,10:04:00,67-22:14:22,1,1,production,0:0
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05T11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05T19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07T20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10T16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0


In [5]:
fullsample_df ['STATE'] = fullsample_df['STATE'].astype('str')

In [6]:
fullsample_df ['BEGIN']= pd.to_datetime(fullsample_df['BEGIN'], errors ='coerce')

In [7]:
fullsample_df['END']= pd.to_datetime(fullsample_df['END'], errors ='coerce')

In [8]:
fullsample_df['REQMEM'] = fullsample_df['REQMEM'].str.extract(r'(\d+)')

In [9]:
fullsample_df['USEDMEM'] = fullsample_df['USEDMEM'].str.extract(r'(\d+)')

In [10]:
fullsample_df['REQTIME'] = fullsample_df['REQTIME'].str.extract(r'(\d+)')

In [11]:
fullsample_df['REQTIME'] = fullsample_df['REQTIME'].str.replace("-", " days ")
fullsample_df['USEDTIME'] = fullsample_df['USEDTIME'].str.replace("-", " days ")
fullsample_df['REQTIME'] = pd.to_timedelta(fullsample_df['REQTIME'])
fullsample_df['USEDTIME'] = pd.to_timedelta(fullsample_df['USEDTIME'])

In [12]:
fullsample_df['UNIT'] = fullsample_df['REQMEM'].str.extract(r'(\D)')
